## Code generation

In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install sympy
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib

In [2]:
# Remove matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# initialize sympy printing (for latex output)
from sympy import init_printing, Symbol
init_printing()

# import functions and classes for compartment models
import sys
sys.path.insert(0, "./compartor") #use local copy
from compartor import *

Here, we illustrate how to export the moment equations in LaTeX format or generate code for simulations.

We consider again the exemplary model of the paper and derive moments equations with the automated function.

In [4]:
x = Content('x')
y = Content('y')
z = Content('z')

from sympy import symbols, log, exp, sin, cos, tan
kM,h = symbols('kM h') # The Michaelis-Menten constant and hill exponent

# define intake distribution
pi_I0 = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss0}(y; \lambda)"), y[0], Symbol("\lambda"))
pi_I1 = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss1}(y; \lambda)"), y[1], Symbol("\lambda"))
pi_I = OutcomeDistribution.CombineIndependent(Symbol("\pi_{Poiss}(y; \lambda)"), pi_I0, pi_I1)
# define fragmentation distribution
pi_F0 = OutcomeDistribution.Uniform(Symbol("\pi_{F0}(y|x)"), y[0], 0, x[0])
pi_F1 = OutcomeDistribution.Uniform(Symbol("\pi_{F1}(y|x)"), y[1], 0, x[1])
pi_F = OutcomeDistribution.CombineIndependent(Symbol("\pi_F(y|x)"), pi_F0, pi_F1)

Birth0         = TransitionClass( [x]       -to> [x+(1,0)],       'k_b',                  name='b_0')
Death0         = TransitionClass( [x]       -to> [x+(-1,0)],       'k_d', g=x[0],    name='d_0')
Birth1         = TransitionClass( [x]       -to> [x+(0,1)],       'k_b',                  name='b_1')
Death1         = TransitionClass( [x]       -to> [x+(0,-1)],       'k_d', g=x[1],    name='d_1')

Intake        = TransitionClass( {}        -to> [y],         'k_I',         pi=pi_I, name='I')
Exit          = TransitionClass( [x]       -to> {},          'k_E',                  name='E')

Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C',                  name='C')
Fragmentation = TransitionClass( [x]       -to> [y] + [x-y], 'k_F', g=x[0]*x[1], pi=pi_F, name='F')

transitions = [Intake, Exit, Coagulation, Fragmentation, Birth0, Death0, Birth1, Death1]
display_transition_classes(transitions)

                     EmptySet ---> [y],  h_I = \pi_{Poiss}(y; \lambda)*k_I    
                               [x] ---> EmptySet,  h_E = k_E*n(x)             
[x] + [y] ---> [x + y],  h_C = k_C*(n(y) - KroneckerDelta(x, y))*n(x)/(Kroneck
                  [x] ---> [x - y] + [y],  h_F = \pi_F(y|x)*k_F*n(x)*x[0]*x[1]
                            [x] ---> [(1, 0) + x],  h_b_0 = k_b*n(x)          
                         [x] ---> [(-1, 0) + x],  h_d_0 = k_d*n(x)*x[0]       
                            [x] ---> [(0, 1) + x],  h_b_1 = k_b*n(x)          
                         [x] ---> [(0, -1) + x],  h_d_1 = k_d*n(x)*x[1]       

                  
                  
erDelta(x, y) + 1)
                  
                  
                  
                  
                  

In [5]:
M = Moment
moments = [
    M(0,0),
    M(0,0)**2,
    M(1,0),
    M(1,0)**2,
    M(0,1),
    M(0,1)**2,
    M(1,1),
]
display(moments)
equations = automated_moment_equations(2, transitions, moments, clna=True)
display_moment_equations(equations)

⎡                    2                            2                           
⎣Moment(0, 0), Moment (0, 0), Moment(1, 0), Moment (1, 0), Moment(0, 1), Momen

 2                    ⎤
t (0, 1), Moment(1, 1)⎦

dfM: DeltaM(0, 0)
monomials: [(1, 1, DeltaM(0, 0))]
contentFun: 1
contentFun_clna: 1
dfMdt: k_I
contentFun: -1
contentFun_clna: -1
dfMdt: -k_E*Moment(0, 0)
contentFun: -1
contentFun_clna: -1
CHECK BOOLS: alpha=[0, 0], beta=[0, 0], boolean_variables=set()
	l_n_Xc: -k_C
	replaced1: [-k_C*(2*(Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + E[Moment(0, 0)]**2)/2]
	replaced2: [-k_C*Moment(0, 0)/2]
dfMdt: k_C*(-(2*Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + Moment(0, 0))/2
contentFun: x[0]*x[1]
contentFun_clna: ((E[Moment(0, 1)]*x[0] + E[Moment(1, 0)]*x[1])*E[Moment(0, 0)] - E[Moment(0, 1)]*E[Moment(1, 0)])/E[Moment(0, 0)]**2
dfMdt: k_F*((Moment(0, 1)*E[Moment(1, 0)] + Moment(1, 0)*E[Moment(0, 1)])*E[Moment(0, 0)] - Moment(0, 0)*E[Moment(0, 1)]*E[Moment(1, 0)])/E[Moment(0, 0)]**2
contentFun: 0
contentFun_clna: 0
dfMdt: 0
contentFun: 0
contentFun_clna: 0
dfMdt: 0
contentFun: 0
contentFun_clna: 0
dfMdt: 0
contentFun: 0
contentFun_clna: 0
dfMdt: 0
dfM: DeltaM(0, 0)**2 + 2*DeltaM(0, 0)*M

dfMdt: k_C*Moment(0, 0)*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**2 + k_C*Moment(0, 1)*E[Moment(1, 0)] - k_C*Moment(0, 1)*E[Moment(1, 0)]/E[Moment(0, 0)] + k_C*Moment(1, 0)*E[Moment(0, 1)] - k_C*Moment(1, 0)*E[Moment(0, 1)]/E[Moment(0, 0)] - k_C*E[Moment(0, 1)]*E[Moment(1, 0)]
contentFun: -x[0]**2*x[1]**2/2
contentFun_clna: (-2*(E[Moment(0, 1)]*x[0] + E[Moment(1, 0)]*x[1])*E[Moment(0, 0)] + 3*E[Moment(0, 1)]*E[Moment(1, 0)])*E[Moment(0, 1)]*E[Moment(1, 0)]/(2*E[Moment(0, 0)]**4)
dfMdt: k_F*(-2*(Moment(0, 1)*E[Moment(1, 0)] + Moment(1, 0)*E[Moment(0, 1)])*E[Moment(0, 0)] + 3*Moment(0, 0)*E[Moment(0, 1)]*E[Moment(1, 0)])*E[Moment(0, 1)]*E[Moment(1, 0)]/(2*E[Moment(0, 0)]**4)
contentFun: x[1]
contentFun_clna: x[1]
dfMdt: k_b*Moment(0, 1)
contentFun: -x[0]*x[1]
contentFun_clna: (-(E[Moment(0, 1)]*x[0] + E[Moment(1, 0)]*x[1])*E[Moment(0, 0)] + E[Moment(0, 1)]*E[Moment(1, 0)])/E[Moment(0, 0)]**2
dfMdt: k_d*(-(Moment(0, 1)*E[Moment(1, 0)] + Moment(1, 0)*E[Moment(0, 1)])*E[Moment(0, 0)]

dfM: DeltaM(1, 1)
monomials: [(1, 1, DeltaM(1, 1))]
contentFun: \lambda**2
contentFun_clna: \lambda**2
dfMdt: \lambda**2*k_I
contentFun: -x[0]*x[1]
contentFun_clna: (-(E[Moment(0, 1)]*x[0] + E[Moment(1, 0)]*x[1])*E[Moment(0, 0)] + E[Moment(0, 1)]*E[Moment(1, 0)])/E[Moment(0, 0)]**2
dfMdt: k_E*(-(Moment(0, 1)*E[Moment(1, 0)] + Moment(1, 0)*E[Moment(0, 1)])*E[Moment(0, 0)] + Moment(0, 0)*E[Moment(0, 1)]*E[Moment(1, 0)])/E[Moment(0, 0)]**2
contentFun: x[0]*y[1] + x[1]*y[0]
contentFun_clna: ((E[Moment(0, 1)]*x[0] + E[Moment(0, 1)]*y[0] + E[Moment(1, 0)]*x[1] + E[Moment(1, 0)]*y[1])*E[Moment(0, 0)] - 2*E[Moment(0, 1)]*E[Moment(1, 0)])/E[Moment(0, 0)]**2
CHECK BOOLS: alpha=[1, 0], beta=[0, 0], boolean_variables=set()
CHECK BOOLS: alpha=[0, 0], beta=[1, 0], boolean_variables=set()
CHECK BOOLS: alpha=[0, 1], beta=[0, 0], boolean_variables=set()
CHECK BOOLS: alpha=[0, 0], beta=[0, 1], boolean_variables=set()
CHECK BOOLS: alpha=[0, 0], beta=[0, 0], boolean_variables=set()
	l_n_Xc: k_C*((E[Moment

dfM: DeltaM(0, 0)**2 + 2*DeltaM(0, 0)*Moment(0, 0)
monomials: [(1, 1, DeltaM(0, 0)**2), (2, Moment(0, 0), DeltaM(0, 0))]
contentFun: 1
contentFun_clna: 1
dfMdt: k_I
contentFun: 1
contentFun_clna: 1
dfMdt: 2*k_I*Moment(0, 0)
contentFun: 1
contentFun_clna: 1
dfMdt: k_E*Moment(0, 0)
contentFun: -1
contentFun_clna: -1
dfMdt: -2*k_E*Moment(0, 0)**2
contentFun: 1
contentFun_clna: 1
CHECK BOOLS: alpha=[0, 0], beta=[0, 0], boolean_variables=set()
	l_n_Xc: k_C
	replaced1: [k_C*(2*(Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + E[Moment(0, 0)]**2)/2]
	replaced2: [k_C*Moment(0, 0)/2]
dfMdt: k_C*((2*Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] - Moment(0, 0))/2
contentFun: -1
contentFun_clna: -1
CHECK BOOLS: alpha=[0, 0], beta=[0, 0], boolean_variables=set()
	l_n_Xc: -2*k_C*Moment(0, 0)
	replaced1: [-k_C*(2*(Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + E[Moment(0, 0)]**2)*Moment(0, 0)]
	replaced2: [-k_C*Moment(0, 0)**2]
dfMdt: k_C*(-(2*Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + M

...

                                                                              
d ⎛           ⎛      2      ⎞⎞          2                                     
──⎝Expectation⎝Moment (0, 1)⎠⎠ = \lambda ⋅k_I + 2⋅\lambda⋅k_I⋅Expectation(Mome
dt                                                                            

                                                                            2 
                                           ⎛      2      ⎞   k_E⋅Expectation (
nt(0, 1)) + \lambda⋅k_I - 2⋅k_E⋅Expectation⎝Moment (0, 1)⎠ + ─────────────────
                                                                     Expectati

                                                                              
Moment(0, 1))                                                                 
───────────── + k_b⋅Expectation(N) + 2⋅k_b⋅Expectation(N⋅Moment(0, 1)) + k_d⋅E
on(N)                                                                         

                                                 

                                                 2                          2 
d                               - k_F⋅Expectation (Moment(0, 1))⋅Expectation (
──(Expectation(Moment(1, 1))) = ──────────────────────────────────────────────
dt                                                                            
                                                                              

                                                 2                            
Moment(1, 0)) - 2⋅(k_C + k_E + 2⋅k_d)⋅Expectation (N)⋅Expectation(Moment(0, 1)
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                ⎛       2                                     
)⋅Expectation(Moment(1, 0)) + 2⋅⎝\lambda ⋅k_I + k_C⋅Expectation(Moment(0, 1))⋅
──────────────────────────────────────────────────

                                                                              
d                                                                            k
──(Expectation(N⋅Moment(1, 0))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I + ─
dt                                                                            
                                                                              

              2                                                               
_C⋅Expectation (N)⋅Expectation(Moment(1, 0))                                  
──────────────────────────────────────────── - k_C⋅Expectation(N)⋅Expectation(
                     2                                                        
                                                                              

                                                                              
                  k_C⋅Expectation(N⋅Moment(1, 0))                             
N⋅Moment(1, 0)) + ─────────────────────────────── 

                                                                              
d                                                                            k
──(Expectation(N⋅Moment(0, 1))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I + ─
dt                                                                            
                                                                              

              2                                                               
_C⋅Expectation (N)⋅Expectation(Moment(0, 1))                                  
──────────────────────────────────────────── - k_C⋅Expectation(N)⋅Expectation(
                     2                                                        
                                                                              

                                                                              
                  k_C⋅Expectation(N⋅Moment(0, 1))                             
N⋅Moment(0, 1)) + ─────────────────────────────── 

d                                                   2                         
──(Expectation(Moment(0, 1)⋅Moment(1, 0))) = \lambda ⋅k_I + \lambda⋅k_I⋅Expect
dt                                                                            

                                                                              
ation(Moment(0, 1)) + \lambda⋅k_I⋅Expectation(Moment(1, 0)) - 2⋅k_E⋅Expectatio
                                                                              

                               k_E⋅Expectation(Moment(0, 1))⋅Expectation(Momen
n(Moment(0, 1)⋅Moment(1, 0)) + ───────────────────────────────────────────────
                                                    Expectation(N)            

t(1, 0))                                                                      
──────── + k_b⋅Expectation(N⋅Moment(0, 1)) + k_b⋅Expectation(N⋅Moment(1, 0)) -
                                                                              

                                             
 2

The LaTeX source of the ODE system can be found and copy-pasted by modifying the Math Render option of the jupyter notebook. This is done by right-clicking on the system and choosing Math Settings > Math Render > Plain Source .

From the closed equations, we can also generate code to simulate the system. Currently, Python or Julia code can be generated.

For a direct code output, the user can rely on the functions:

In [8]:
julia_code = generate_julia_code(equations, function_name="IECF2BD_bis")
print(julia_code)

# evaluate ODEs
function IECF2BD_bis_ODEs(dM, M, parameters, t)
  c0 = parameters[:\lambda] # \lambda
  c3 = parameters[:kC] # kC
  c6 = parameters[:kE] # kE
  c1 = parameters[:kF] # kF
  c2 = parameters[:kI] # kI
  c4 = parameters[:kb] # kb
  c5 = parameters[:kd] # kd
  # Number of Compartments (N)
  dM[1] = c2+1/2*c3*M[1]-c6*M[1]-1/2*c3*M[1]^2+c1*M[5]*M[3]/M[1]
  # N^2
  dM[2] = (1/2*M[1]^2*(2*c2+c3*M[1]^2-c3*M[1]-4*c6*M[2]+2*c3*M[1]^3+2*c3*M[2]+2*c6*M[1]+4*c2*M[1]-4*c3*M[2]*M[1])+c1*(M[5]*M[3]+2*M[9]*M[3]+2*M[8]*M[5])*M[1]-2*c1*M[2]*M[5]*M[3])/M[1]^2
  # Total Mass of species 1
  dM[3] = c0*c2+c4*M[1]-c6*M[3]-c5*M[3]
  # M1^2
  dM[4] = c0*c2+c2*c0^2+c4*M[1]+c5*M[3]-2*c6*M[4]-2*c5*M[4]+2*c4*M[8]+c6*M[3]^2/M[1]+2*c0*c2*M[3]
  # Total Mass of species 2
  dM[5] = c0*c2+c4*M[1]-c6*M[5]-c5*M[5]
  # M2^2
  dM[6] = c0*c2+c2*c0^2+c4*M[1]+c5*M[5]-2*c6*M[6]-2*c5*M[6]+2*c4*M[9]+c6*M[5]^2/M[1]+2*c0*c2*M[5]
  # Sum of squared content of species 1
  dM[7] = 1/2*(2*M[1]^3*(c2*c0^2+c4*M[5]+c4*M[3]+c